# Gradient Descent for Univariate Linear Regression

## Import Data and Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

In this tutorial, we will try to build a simple linear regression model from scratch to predict `Sales` as *dependent variable* (*target*) using `TV Price` as *independent variable* (*predictor*)

Recall that Univariate Linear Regression is just a simple linear equation $ f(x) = w.x + b $. The idea of univariate linear regression is as simple as finding the approximate function $ f(x) $ which can minimize the error between *target* and *prediction*.

In [5]:
data = pd.read_csv("data/tvmarketing.csv")
X = data.TV.values
y = data.Sales.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
data.head(3)

,TV,Sales
0,230.1,22.1
1,44.5,10.4
2,17.2,9.3


In order to make the computation much faster, we can scale the `TV` column. In this tutorial, I used a StandardScaler from scikit-learn. This scaling process sometimes called as `z-score standardization` which we can see as follow:

<br>
<center>
    $ x_{scale} = \frac{x^{i} - \mu}{\sigma}$
</center>

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, 1))
print(scaler.mean_)

[150.019375]


In [8]:
X_train = X_train.reshape((X_train.shape[0]))

## Gradient Descent Algorithm

> Problem: How to find $w$ and $b$ so $ f(x) $ can minimize the error?

To solve this problem, we can use an optimization algorithm which is called as **Gradient Descent**.

I won't talk much about this algorithm, but this are the core steps of the algorithm:

`Loop`:
    
1. Initialize $w$ and $b$ with a value


2. Compute cost function over all training samples
<center>
$
    \mathcal{L}(\hat{y}^{(i)}, y^{(i)}) = (\hat{y}^{(i)} - y^{i})^{2} = (f_{w, b}(x^{(i)}) - y^{(i)})^{2} = (w.x^{(i)} + b - y^{(i)})^{2} \\
    \mathcal{J}(w, b) = \frac{1}{2m}\sum_{i=1} ^{m} \mathcal{L}(\hat{y}^{(i)}, y^{(i)}) = \frac{1}{2m}\sum_{i=1} ^{m} (w.x^{(i)} + b - y^{(i)})^{2}
$
</center>
    

3. Update $w$ and $b$ simultaneously
<center>
$
    w = w - \alpha\frac{\partial{\mathcal{J}(w, b)}}{\partial{w}} \\
    b = b - \alpha\frac{\partial{\mathcal{J}(w, b)}}{\partial{b}}
$  
</center>


4. Back to 1st step until the algorithm converged

In [9]:
X_train.shape, y_train.shape

((160,), (160,))

In [10]:
def compute_cost(X, y, w, b):
    m = X.shape[0]
    total_cost = 0
    for i in range(m):
        f_wb = w * X[i] + b
        cost = (f_wb - y[i]) ** 2
        total_cost += cost
    return 1 / (2 * m) * total_cost

In [11]:
compute_cost(X_train, y_train, 1.5, 3)

69.82458592927999

In [163]:
def compute_gradients(X, y, w, b):
    m = X.shape[0]
    dj_dw = 0
    dj_db = 0
    for i in range(m):
        x_i = X[i]
        y_i = y[i]
        f_i = (w * x_i + b - y_i)
        dj_dw_i = (f_i - y_i) * x_i
        dj_db_i = (f_i - y_i)
        dj_dw += dj_dw_i
        dj_db += dj_db_i
    dj_dw /= m
    dj_db /= m
    return dj_dw, dj_db

In [164]:
compute_gradients(X_train, y_train, w=1.5, b=3)

(-6.331385427626634, -25.2)

In [165]:
def one_step_gradient_descent(X, y, w_init, b_init, learning_rate, compute_gradients=compute_gradients):
    w = w_init
    b = b_init
    dj_dw, dj_db = compute_gradients(X, y, w=w_init, b=b_init)
    w = w - learning_rate * dj_dw
    b = b - learning_rate * dj_db
    return w, b

In [166]:
one_step_gradient_descent(X_train, y_train, 1.5, 3, 0.01)

(1.5633138542762663, 3.252)

In [180]:
def gradient_descent(X, y, w_init, b_init, num_iters, learning_rate=1e-6, compute_cost=compute_cost):
    losses = []
    params_hist = []
    w = w_init
    b = b_init
    for iteration in range(num_iters):
        loss = compute_cost(X, y, w, b)
        if iteration != 0 and loss > losses[iteration - 1]:
            print("[STOP] The Algorithm has converged")
            break
        w, b = one_step_gradient_descent(X, y, w, b, learning_rate)
        losses.append(loss)
        params_hist.append([w, b])
        if iteration % 1000 == 0:
            print(f"Iteration: {iteration:05d} | Loss: {loss:.3f} | W: {w:.3f} | b = {b:.3f}")
    return losses, params_hist

In [181]:
losses, params_hist = gradient_descent(X_train, y_train, w_init=0, b_init=0, num_iters=1000000)

Iteration: 00000 | Loss: 112.373 | W: 0.000 | b = 0.000
Iteration: 01000 | Loss: 111.945 | W: 0.008 | b = 0.028
Iteration: 02000 | Loss: 111.519 | W: 0.016 | b = 0.056
Iteration: 03000 | Loss: 111.094 | W: 0.023 | b = 0.085
Iteration: 04000 | Loss: 110.670 | W: 0.031 | b = 0.113
Iteration: 05000 | Loss: 110.248 | W: 0.039 | b = 0.141
Iteration: 06000 | Loss: 109.826 | W: 0.047 | b = 0.169
Iteration: 07000 | Loss: 109.406 | W: 0.055 | b = 0.197
Iteration: 08000 | Loss: 108.988 | W: 0.062 | b = 0.225
Iteration: 09000 | Loss: 108.570 | W: 0.070 | b = 0.253
Iteration: 10000 | Loss: 108.154 | W: 0.078 | b = 0.281
Iteration: 11000 | Loss: 107.739 | W: 0.086 | b = 0.309
Iteration: 12000 | Loss: 107.325 | W: 0.093 | b = 0.336
Iteration: 13000 | Loss: 106.913 | W: 0.101 | b = 0.364
Iteration: 14000 | Loss: 106.502 | W: 0.109 | b = 0.392
Iteration: 15000 | Loss: 106.092 | W: 0.117 | b = 0.420
Iteration: 16000 | Loss: 105.683 | W: 0.124 | b = 0.448
Iteration: 17000 | Loss: 105.276 | W: 0.132 | b 

Iteration: 148000 | Loss: 61.560 | W: 1.077 | b = 3.879
Iteration: 149000 | Loss: 61.292 | W: 1.084 | b = 3.904
Iteration: 150000 | Loss: 61.026 | W: 1.091 | b = 3.928
Iteration: 151000 | Loss: 60.761 | W: 1.098 | b = 3.952
Iteration: 152000 | Loss: 60.496 | W: 1.104 | b = 3.977
Iteration: 153000 | Loss: 60.232 | W: 1.111 | b = 4.001
Iteration: 154000 | Loss: 59.970 | W: 1.118 | b = 4.025
Iteration: 155000 | Loss: 59.708 | W: 1.124 | b = 4.049
Iteration: 156000 | Loss: 59.447 | W: 1.131 | b = 4.073
Iteration: 157000 | Loss: 59.186 | W: 1.138 | b = 4.097
Iteration: 158000 | Loss: 58.927 | W: 1.145 | b = 4.121
Iteration: 159000 | Loss: 58.669 | W: 1.151 | b = 4.146
Iteration: 160000 | Loss: 58.411 | W: 1.158 | b = 4.170
Iteration: 161000 | Loss: 58.155 | W: 1.165 | b = 4.194
Iteration: 162000 | Loss: 57.899 | W: 1.171 | b = 4.218
Iteration: 163000 | Loss: 57.644 | W: 1.178 | b = 4.242
Iteration: 164000 | Loss: 57.390 | W: 1.185 | b = 4.266
Iteration: 165000 | Loss: 57.137 | W: 1.191 | b 

Iteration: 296000 | Loss: 30.756 | W: 2.007 | b = 7.225
Iteration: 297000 | Loss: 30.601 | W: 2.012 | b = 7.246
Iteration: 298000 | Loss: 30.446 | W: 2.018 | b = 7.267
Iteration: 299000 | Loss: 30.293 | W: 2.024 | b = 7.288
Iteration: 300000 | Loss: 30.140 | W: 2.030 | b = 7.309
Iteration: 301000 | Loss: 29.987 | W: 2.036 | b = 7.330
Iteration: 302000 | Loss: 29.835 | W: 2.041 | b = 7.351
Iteration: 303000 | Loss: 29.684 | W: 2.047 | b = 7.372
Iteration: 304000 | Loss: 29.533 | W: 2.053 | b = 7.392
Iteration: 305000 | Loss: 29.383 | W: 2.059 | b = 7.413
Iteration: 306000 | Loss: 29.234 | W: 2.064 | b = 7.434
Iteration: 307000 | Loss: 29.085 | W: 2.070 | b = 7.455
Iteration: 308000 | Loss: 28.937 | W: 2.076 | b = 7.475
Iteration: 309000 | Loss: 28.789 | W: 2.082 | b = 7.496
Iteration: 310000 | Loss: 28.642 | W: 2.087 | b = 7.517
Iteration: 311000 | Loss: 28.496 | W: 2.093 | b = 7.537
Iteration: 312000 | Loss: 28.350 | W: 2.099 | b = 7.558
Iteration: 313000 | Loss: 28.205 | W: 2.105 | b 

Iteration: 444000 | Loss: 13.873 | W: 2.808 | b = 10.111
Iteration: 445000 | Loss: 13.795 | W: 2.813 | b = 10.129
Iteration: 446000 | Loss: 13.718 | W: 2.818 | b = 10.147
Iteration: 447000 | Loss: 13.642 | W: 2.823 | b = 10.165
Iteration: 448000 | Loss: 13.565 | W: 2.828 | b = 10.183
Iteration: 449000 | Loss: 13.490 | W: 2.833 | b = 10.201
Iteration: 450000 | Loss: 13.414 | W: 2.838 | b = 10.219
Iteration: 451000 | Loss: 13.339 | W: 2.843 | b = 10.237
Iteration: 452000 | Loss: 13.265 | W: 2.848 | b = 10.255
Iteration: 453000 | Loss: 13.191 | W: 2.853 | b = 10.273
Iteration: 454000 | Loss: 13.117 | W: 2.858 | b = 10.291
Iteration: 455000 | Loss: 13.044 | W: 2.863 | b = 10.309
Iteration: 456000 | Loss: 12.971 | W: 2.868 | b = 10.326
Iteration: 457000 | Loss: 12.898 | W: 2.873 | b = 10.344
Iteration: 458000 | Loss: 12.826 | W: 2.878 | b = 10.362
Iteration: 459000 | Loss: 12.755 | W: 2.883 | b = 10.380
Iteration: 460000 | Loss: 12.683 | W: 2.888 | b = 10.398
Iteration: 461000 | Loss: 12.61

Iteration: 591000 | Loss: 6.540 | W: 3.495 | b = 12.584
Iteration: 592000 | Loss: 6.515 | W: 3.499 | b = 12.599
Iteration: 593000 | Loss: 6.490 | W: 3.503 | b = 12.615
Iteration: 594000 | Loss: 6.465 | W: 3.508 | b = 12.630
Iteration: 595000 | Loss: 6.440 | W: 3.512 | b = 12.646
Iteration: 596000 | Loss: 6.416 | W: 3.516 | b = 12.662
Iteration: 597000 | Loss: 6.392 | W: 3.521 | b = 12.677
Iteration: 598000 | Loss: 6.369 | W: 3.525 | b = 12.693
Iteration: 599000 | Loss: 6.345 | W: 3.529 | b = 12.708
Iteration: 600000 | Loss: 6.322 | W: 3.533 | b = 12.724
Iteration: 601000 | Loss: 6.299 | W: 3.538 | b = 12.739
Iteration: 602000 | Loss: 6.277 | W: 3.542 | b = 12.754
Iteration: 603000 | Loss: 6.255 | W: 3.546 | b = 12.770
Iteration: 604000 | Loss: 6.233 | W: 3.551 | b = 12.785
Iteration: 605000 | Loss: 6.211 | W: 3.555 | b = 12.801
Iteration: 606000 | Loss: 6.190 | W: 3.559 | b = 12.816
Iteration: 607000 | Loss: 6.168 | W: 3.563 | b = 12.831
Iteration: 608000 | Loss: 6.148 | W: 3.568 | b =

In [182]:
w_final, b_final = params_hist[-1]

In [183]:
w_final, b_final

(3.915697279564234, 14.100016440791467)

In [187]:
from sklearn.metrics import r2_score
def evaluate(X_test, y_test, w_final, b_final):
    m = X_test.shape[0]
    predictions = []
    for i in range(m):
        pred = w_final * X_test[i] + b_final
        predictions.append(pred)
    print("R2 Score:", r2_score(y_test, predictions))
    return predictions

In [185]:
X_test = scaler.transform(X_test.reshape(-1, 1))
X_test = X_test.reshape((X_test.shape[0]))
X_test

array([ 0.15781217,  0.53925283,  1.69783431, -1.64363349,  0.83513672,
       -0.89025846,  0.79354661, -1.18851892,  0.86009078,  0.29803023,
       -1.40835233, -1.11484502,  1.00387371, -1.71849568, -0.12500054,
        0.23980408, -1.69591819,  0.56539519, -0.88788188,  1.03833409,
        0.94445928, -0.97700354, -1.33111357,  1.19875306, -0.96274407,
       -1.14692882,  0.75433309, -0.12737712, -0.74647553, -1.68284702,
        0.59153754, -0.96036749,  0.58084294, -1.58184248,  1.55286309,
        1.04784039, -1.20871983,  1.46968288, -0.38998892, -1.45588388])

### Testing

In [188]:
predictions = evaluate(X_test, y_test, w_final=w_final, b_final=b_final)

R2 Score: 0.676695871172363


Using simple simple gradient descent algorithm, we can reach R2-Score 67%. This means, 67% of our *dependete*